# 🌌 HI Map Simulation

#### Pragun Nepal  
#### June 25, 2025

---

This notebook contains the code for running the HI map Simulation (with and without axes). The full video of the simulations have been uploaded in my YouTube channel - https://www.youtube.com/@PragunNepal

All the data for this simulation have been obatined from the coupled N-body, FoF-Halo-Finder, and ReionYuga Repositories by Dr. Rajesh Mondal (GitHub : rajeshmondal18 - https://github.com/rajeshmondal18 )

---

## 📚 Table of Contents

1. [Code for HI Map Simulation](#Code_for_N-body_Simulation)
2. [Code for HI Map Simulation (with axes)](#Code_for_N-body_Simulation_axes)
3. [Conclusion](#conclusion)

---


## ⚙️ <a name="Code_for_N-body_Simulation"></a>1. Code for HI map Simulation (without axes)

```python
import os
import re
import glob
import numpy as np
import pyvista as pv
from math import ceil

#Extracting File Data and Information
i_hi_map = 'HI_map_*'      #Pattern for HI map binary files
i_pk     = 'pk.ionzs*_*'     #Pattern for pk.ionz metadata files
output_filename = 'HImap_Simulation.mp4'  #Output movie filename
vmax = 100.0                #Max value for volume rendering color scale
initial_spin_duration_sec = 10
frame_rate = 30
window_size = (3840, 2160)  #resolution - 4K here
min_total_duration_sec = 60  #Minimum Video Duration


file_list = []
for fn in glob.glob(i_hi_map):
    m = re.match(r'HI_map_(?P<z>\d+\.?\d*)$', os.path.basename(fn))
    if not m:
        continue
    z = float(m.group('z'))
    file_list.append((z, fn))
file_list.sort(reverse=True, key=lambda x: x[0])

#Extracting Data
def load_frames():
    frames = []
    for z, fn in file_list:
        Tb = 22.0 * np.sqrt((1 + z) / 7.0)
        xhi = None
        for pf in glob.glob(i_pk):
            mm = re.match(r'pk\.ionzs(?P<xhi>\d+\.?\d*)_(?P<z2>\d+\.?\d*)$', os.path.basename(pf)) #Here, since I have ionzs type files, im extracting neutral fraction value from the filename of these files
            if mm and abs(float(mm.group('z2')) - z) < 1e-6:
                xhi = float(mm.group('xhi'))
                break
        if xhi is None:
            continue
        with open(fn, 'rb') as f:
            mx, my, mz = np.fromfile(f, count=3, dtype=np.int32)
            data = np.fromfile(f, count=mx * my * mz, dtype=np.float32)
        arr = data.reshape((mx, my, mz), order='C') * (Tb * xhi)
        arr = np.clip(arr, 0, vmax)
        frames.append((z, xhi, arr))
    return frames

frames = load_frames()
if not frames:
    raise RuntimeError("No valid HI_map + pk.ionz pairs found.")

nx, ny, nz = frames[0][2].shape
grid_template = pv.ImageData(
    dimensions=(nx + 1, ny + 1, nz + 1),
    spacing=(1, 1, 1),
    origin=(0, 0, 0)
)

#Plotting Setup
plotter = pv.Plotter(off_screen=True, window_size=window_size)
plotter.open_movie(output_filename, framerate=frame_rate)
plotter.set_background("black")

#Initial Setup
grid = grid_template.copy()
grid.cell_data['values'] = frames[0][2].flatten(order='F')
vol = plotter.add_volume(
    grid,
    scalars='values',
    cmap='plasma',
    clim=[0, vmax],
    opacity='linear',
    shade=True,
    show_scalar_bar=False
)

plotter.add_scalar_bar(
    title=r'T(beta) x(HI) [mK]',
    title_font_size=30,
    label_font_size=26,
    vertical=False,
    position_x=0.25,
    position_y=0.02,
    width=0.5,
    height=0.04,
    color='white'
)

#Camera Setup
azim0 = 30
elev0 = 3
plotter.camera_position = 'iso'
plotter.camera.elevation = elev0
plotter.camera.azimuth = azim0
plotter.camera.zoom(0.6)

#Animation Setup
N = len(frames)
initial_spin_frames = initial_spin_duration_sec * frame_rate
remaining_frames = max(min_total_duration_sec * frame_rate - initial_spin_frames, 1)
frames_per_step = ceil(remaining_frames / (N - 1)) if N > 1 else 1
rotation_speed = 0.2
angle = 0

#Initial Settings
for _ in range(initial_spin_frames):
    angle += rotation_speed
    grid = grid_template.copy()
    grid.cell_data['values'] = frames[0][2].flatten(order='F')
    plotter.remove_actor(vol)
    vol = plotter.add_volume(
        grid,
        scalars='values',
        cmap='plasma',
        clim=[0, vmax],
        opacity='linear',
        shade=True,
        show_scalar_bar=False
    )
    info_text = plotter.add_text(
        f'z={frames[0][0]:.3f}\nxHI={frames[0][1]:.3f}',
        position='lower_right',
        font_size=20,
        color='white'
    )
    plotter.camera.azimuth = azim0 + angle
    plotter.camera.elevation = elev0
    plotter.render()
    plotter.write_frame()
    plotter.remove_actor(info_text)

#Interpolation Between Frames
for i in range(len(frames) - 1):
    z1, xhi1, arr1 = frames[i]
    z2, xhi2, arr2 = frames[i + 1]

    for j in range(frames_per_step):
        t = j / frames_per_step
        z_interp = (1 - t) * z1 + t * z2
        xhi_interp = (1 - t) * xhi1 + t * xhi2
        arr_interp = (1 - t) * arr1 + t * arr2

        grid = grid_template.copy()
        grid.cell_data['values'] = arr_interp.flatten(order='F')
        plotter.remove_actor(vol)
        vol = plotter.add_volume(
            grid,
            scalars='values',
            cmap='plasma',
            clim=[0, vmax],
            opacity='linear',
            shade=True,
            show_scalar_bar=False
        )

        info_text = plotter.add_text(
            f'z={z_interp:.3f}\nxHI={xhi_interp:.3f}',
            position='lower_right',
            font_size=20,
            color='white'
        )

        angle += rotation_speed
        plotter.camera.azimuth = azim0 + angle
        plotter.camera.elevation = elev0

        plotter.render()
        plotter.write_frame()
        plotter.remove_actor(info_text)

plotter.close()
print(f"\u2705 Animation saved as '{output_filename}' ")
```
---

## ⚙️ <a name="Code_for_N-body_Simulation_axes"></a>2. Code for HI map Simulation (with axes)

```python
import os
import re
import glob
import numpy as np
import pyvista as pv
from math import ceil

#Configuration
i_hi_map = 'HI_map_*'      #Pattern for HI map binary files
i_pk     = 'pk.ionzs*_*'     #Pattern for pk.ionz metadata files
output_filename = 'HImap_Simulation_with_axes.mp4'  #Output movie filename
vmax = 100.0                #Max value for volume rendering color scale
initial_spin_duration_sec = 10
frame_rate = 30
window_size = (3840, 2160)  #Resolution - 4K here
min_total_duration_sec = 60  #Minimum Video Duration

#Extracting Data from File and Filename
file_list = []
for fn in glob.glob(i_hi_map):
    m = re.match(r'HI_map_(?P<z>\d+\.?\d*)$', os.path.basename(fn))
    if not m:
        continue
    z = float(m.group('z'))
    file_list.append((z, fn))
file_list.sort(reverse=True, key=lambda x: x[0])

# === Data Loader ===
def load_frames():
    frames = []
    for z, fn in file_list:
        Tb = 22.0 * np.sqrt((1 + z) / 7.0)
        xhi = None
        for pf in glob.glob(i_pk):
            mm = re.match(r'pk\.ionzs(?P<xhi>\d+\.?\d*)_(?P<z2>\d+\.?\d*)$', os.path.basename(pf)) #Here, since I have ionzs type files, im extracting neutral fraction value from the filename of these files
            if mm and abs(float(mm.group('z2')) - z) < 1e-6:
                xhi = float(mm.group('xhi'))
                break
        if xhi is None:
            continue
        with open(fn, 'rb') as f:
            mx, my, mz = np.fromfile(f, count=3, dtype=np.int32)
            data = np.fromfile(f, count=mx * my * mz, dtype=np.float32)
        arr = data.reshape((mx, my, mz), order='C') * (Tb * xhi)
        arr = np.clip(arr, 0, vmax)
        frames.append((z, xhi, arr))
    return frames

frames = load_frames()
if not frames:
    raise RuntimeError("No valid HI_map + pk.ionzs pairs found.")

nx, ny, nz = frames[0][2].shape
grid_template = pv.ImageData(
    dimensions=(nx + 1, ny + 1, nz + 1),
    spacing=(1, 1, 1),
    origin=(0, 0, 0)
)

#Plotter Setup
plotter = pv.Plotter(off_screen=True, window_size=window_size)
plotter.open_movie(output_filename, framerate=frame_rate)
plotter.set_background("black")

plotter.show_grid(
    color='cyan',
    location='all',
    bold=True,
    grid=True,
    n_xlabels=int(nx / 0.25),
    n_ylabels=int(ny / 0.25),
    n_zlabels=int(nz / 0.25),
)
plotter.add_axes(line_width=3, color='cyan')

#Initial Setup
grid = grid_template.copy()
grid.cell_data['values'] = frames[0][2].flatten(order='F')
vol = plotter.add_volume(
    grid,
    scalars='values',
    cmap='plasma',
    clim=[0, vmax],
    opacity='linear',
    shade=True,
    show_scalar_bar=False
)

plotter.add_scalar_bar(
    title=r'T(beta) x(HI) [mK]',
    title_font_size=30,
    label_font_size=26,
    vertical=False,
    position_x=0.25,
    position_y=0.02,
    width=0.5,
    height=0.04,
    color='white'
)

#Camera Setup
azim0 = 30
elev0 = 3
plotter.camera_position = 'iso'
plotter.camera.elevation = elev0
plotter.camera.azimuth = azim0
plotter.camera.zoom(0.6)

#Animation Setup
N = len(frames)
initial_spin_frames = initial_spin_duration_sec * frame_rate
remaining_frames = max(min_total_duration_sec * frame_rate - initial_spin_frames, 1)
frames_per_step = ceil(remaining_frames / (N - 1)) if N > 1 else 1
rotation_speed = 0.2
angle = 0

#Initial Setup
for _ in range(initial_spin_frames):
    angle += rotation_speed
    grid = grid_template.copy()
    grid.cell_data['values'] = frames[0][2].flatten(order='F')
    plotter.remove_actor(vol)
    vol = plotter.add_volume(
        grid,
        scalars='values',
        cmap='plasma',
        clim=[0, vmax],
        opacity='linear',
        shade=True,
        show_scalar_bar=False
    )
    info_text = plotter.add_text(
        f'z={frames[0][0]:.3f}\nxHI={frames[0][1]:.3f}',
        position='lower_right',
        font_size=20,
        color='white'
    )
    plotter.camera.azimuth = azim0 + angle
    plotter.camera.elevation = elev0
    plotter.render()
    plotter.write_frame()
    plotter.remove_actor(info_text)

#Interpolation
for i in range(len(frames) - 1):
    z1, xhi1, arr1 = frames[i]
    z2, xhi2, arr2 = frames[i + 1]

    for j in range(frames_per_step):
        t = j / frames_per_step
        z_interp = (1 - t) * z1 + t * z2
        xhi_interp = (1 - t) * xhi1 + t * xhi2
        arr_interp = (1 - t) * arr1 + t * arr2

        grid = grid_template.copy()
        grid.cell_data['values'] = arr_interp.flatten(order='F')
        plotter.remove_actor(vol)
        vol = plotter.add_volume(
            grid,
            scalars='values',
            cmap='plasma',
            clim=[0, vmax],
            opacity='linear',
            shade=True,
            show_scalar_bar=False
        )

        info_text = plotter.add_text(
            f'z={z_interp:.3f}\nxHI={xhi_interp:.3f}',
            position='lower_right',
            font_size=20,
            color='white'
        )

        angle += rotation_speed
        plotter.camera.azimuth = azim0 + angle
        plotter.camera.elevation = elev0

        plotter.render()
        plotter.write_frame()
        plotter.remove_actor(info_text)

plotter.close()
print(f"\u2705 Animation saved as '{output_filename}' ")

```
---

## 🔭 <a name="conclusion"></a>3. Conclusion

The code has been made general, hence its easily customisable. Please make necessary changes to the code as per your needs. 

©Pragun Nepal, 2025

Contact Me :
- Email : pragunnepal@gmail.com
- Email : pragun23@iisertvm.ac.in
- GitHub : PragunNepal
- LinkedIn : Pragun Nepal